In [2]:
# import numpy as np
# import pandas as pd
import os
# import sys
# import future.utils
# import warnings
# import traceback
# import logging
import json

In [3]:
#open a spaghetti plot
def show_evolution(e, g, p_media, media_op, mi=1000000, nr=1):
    from IPython.display import Image
    return Image(filename='plots/media mo{} p{} e{} g{} gm{} mi{} nr{}.png'.format(media_op, p, e, g, g, mi, nr)) 

In [4]:
def enc(e, g, p_media, media_op, mi=1000000, nr=1):
    file = open("aggregate/enc media mo{} p{} e{} g{} gm{} mi{}.csv".format(media_op, p, e, g, g, mi), "r")
    values = list((file.readlines()[0]).split(','))
    file.close()
    return float(values[nr-1])

In [5]:
def final_clusters(e, g, p_media, media_op, mi=1000000, nr=1):
    import json
    # Opening JSON file
    with open("aggregate/final_clusters media mo{} p{} e{} g{} gm{} mi{}.json".format(media_op, p, e, g, g, mi)) as json_file:
        data = json.load(json_file)
    clusters = data['{}'.format(nr)]
    return clusters

In [6]:
def final_opinions(e, g, p_media, media_op, mi=1000000):   
    filename = "aggregate/final_iterations media mo{} p{} e{} g{} gm{} mi{} stofacendoleprove.json".format(media_op, p, e, g, g, mi)
    filenamedue = "aggregate/final_iterations media mo{} p{} e{} g{} gm{} mi{}.json".format(media_op, p, e, g, g, mi)
    try:
        json_file = open(filename)    
        data = json.load(json_file)
        json_file_due = open(filenamedue)
        datadue = json.load(json_file_due)
        json_file.close()
        json_file_due.close()
        if(len(datadue)==100):   
            print("eliminating")
            os.remove(filenamedue)
            os.rename(filename, filenamedue)
    except Exception as err:
        print(err)

In [7]:
for e in [0.1, 0.2, 0.3, 0.4, 0.5, 1.0]:
    for g in [0.0, 0.5, 0.75, 1.0, 1.25, 1.5]:
        for p in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
            final_opinions(e, g, p, [0.0])

[Errno 2] No such file or directory: 'aggregate/final_iterations media mo[0.0] p0.0 e0.1 g0.0 gm0.0 mi1000000 stofacendoleprove.json'
[Errno 2] No such file or directory: 'aggregate/final_iterations media mo[0.0] p0.1 e0.1 g0.0 gm0.0 mi1000000 stofacendoleprove.json'
[Errno 2] No such file or directory: 'aggregate/final_iterations media mo[0.0] p0.2 e0.1 g0.0 gm0.0 mi1000000 stofacendoleprove.json'
[Errno 2] No such file or directory: 'aggregate/final_iterations media mo[0.0] p0.3 e0.1 g0.0 gm0.0 mi1000000 stofacendoleprove.json'
[Errno 2] No such file or directory: 'aggregate/final_iterations media mo[0.0] p0.4 e0.1 g0.0 gm0.0 mi1000000 stofacendoleprove.json'
[Errno 2] No such file or directory: 'aggregate/final_iterations media mo[0.0] p0.5 e0.1 g0.0 gm0.0 mi1000000 stofacendoleprove.json'
[Errno 2] No such file or directory: 'aggregate/final_iterations media mo[0.0] p0.0 e0.1 g0.5 gm0.5 mi1000000 stofacendoleprove.json'
[Errno 2] No such file or directory: 'aggregate/final_iteratio